# Web Server Log Analysis with Apache Spark

#![Webserver Logo](https://mpng.pngfly.com/20180816/btu/kisspng-apache-http-server-web-server-computer-servers-ins-design-web-services-product-ux-design-drupal-so-5b757d80b4d064.6455601615344264967406.jpg) 


This project will demonstrate to perform web server log analysis with Apache Spark.

What are Weblogs? Weblogs are where web server (like apache) records events like visitors to your site and problems it's encountered. Your web server records all the visitors to your site. There you can see what files users are accessing, how the web server responded to requests, and other information like what kind of web browsers visitors are using, etc.


Log data comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more.

Sample Weblog Data:
in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839

Format of weblogs

| field         | meaning                                                                                                      |
| ------------- | -------------------------------------------------------------------------------------------------------------|
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available).                                             |
| _rfc931_      | The remote logname of the user. We don't really care about this field.                                       |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server.We don't really care about this field.  |
| _[date]_      | The date and time of the request.                                                                            |
| _"request"_   | The request, exactly as it came from the browser or client.                                                  |
| _status_      | The HTTP status code the server sent back to the client.                                                     |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.                                            |


KPI : Parse weblogs into structured format ('host', 'dateTime', 'request_string', 'status', 'content_size')

Then do following analysis:

1. we will find what are the average, minimum, and maximum content sizes of web logs.

2. To find which HTTP status values appear in the data and how many times.

3. Find the hosts that have accessed the server frequently. As with the response code analysis We then filter the result based on the count of accesses by each host. Then, we select the 'host' column and show few elements from the result.

4. Find the top paths/request-strings (URIs) in the log.

5. Find top ten error paths which did not have return code 200?

6. How many unique hosts are there in the entire log?

7. Number of Unique Daily Hosts

8. Number of Daily Requests per Unique Host

9. Counting 404 Response Codes

10. Find top twenty request_string/paths that generate the most 404 errors.

11. Visualizing the 404 Errors by Day

12. Top Five Days for 404 Errors

#Loading data into spark dataframes

In [3]:
%python

spark.conf.set('spark.sql.shuffle.partitions','8')

weblog_rawdata = spark.read.text("/FileStore/tables/Web_log")

display(weblog_rawdata)


value
"in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] ""GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0"" 200 1839"
"uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] ""GET / HTTP/1.0"" 304 0"
"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/ksclogo-medium.gif HTTP/1.0"" 304 0"
"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/MOSAIC-logosmall.gif HTTP/1.0"" 304 0"
"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/USA-logosmall.gif HTTP/1.0"" 304 0"
"ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] ""GET /images/launch-logo.gif HTTP/1.0"" 200 1713"
"uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] ""GET /images/WORLD-logosmall.gif HTTP/1.0"" 304 0"
"slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] ""GET /history/skylab/skylab.html HTTP/1.0"" 200 1687"
"piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] ""GET /images/launchmedium.gif HTTP/1.0"" 200 11853"
"slppp6.intermind.net - - [01/Aug/1995:00:00:11 -0400] ""GET /history/skylab/skylab-small.gif HTTP/1.0"" 200 9202"


## Total number of records in weblogs

In [5]:
weblog_rawdata.count()


Out[3]: 1569898

##Parsing the weblogs (Splitting the Data to multiple columns from Single Column)

In [7]:
from pyspark.sql.functions import col,regexp_extract

weblog_parsed = weblog_rawdata.withColumn('host',regexp_extract(col("value"),'^([^\s]+\s)',1)).withColumn('dateTime', regexp_extract(col("value"),'\[(\w.*?)\]',1)).withColumn('request_string', regexp_extract(col("value"),'^.*"\w+\s+([^\s]+)\s+HTTP.*"',1)).withColumn('status', regexp_extract(col("value"),'^.*"\s+([^\s]+)',1).cast('int')).withColumn('content_size', regexp_extract(col("value"),'^.*\s+(\d+)$',1).cast('int')).drop('value')

display(weblog_parsed)

host,dateTime,request_string,status,content_size
in24.inetnebr.com,01/Aug/1995:00:00:01 -0400,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
uplherc.upl.com,01/Aug/1995:00:00:07 -0400,/,304,0
uplherc.upl.com,01/Aug/1995:00:00:08 -0400,/images/ksclogo-medium.gif,304,0
uplherc.upl.com,01/Aug/1995:00:00:08 -0400,/images/MOSAIC-logosmall.gif,304,0
uplherc.upl.com,01/Aug/1995:00:00:08 -0400,/images/USA-logosmall.gif,304,0
ix-esc-ca2-07.ix.netcom.com,01/Aug/1995:00:00:09 -0400,/images/launch-logo.gif,200,1713
uplherc.upl.com,01/Aug/1995:00:00:10 -0400,/images/WORLD-logosmall.gif,304,0
slppp6.intermind.net,01/Aug/1995:00:00:10 -0400,/history/skylab/skylab.html,200,1687
piweba4y.prodigy.com,01/Aug/1995:00:00:10 -0400,/images/launchmedium.gif,200,11853
slppp6.intermind.net,01/Aug/1995:00:00:11 -0400,/history/skylab/skylab-small.gif,200,9202


## 1. Find what are the average, minimum, and maximum content sizes of web logs.

In [9]:
from pyspark.sql.functions import avg,max,min

content_stat = weblog_parsed.select(min('content_size'),max('content_size'),avg('content_size'))


display(content_stat)

min(content_size),max(content_size),avg(content_size)
0,3421948,17244.9678759674


##2. HTTP Status Analysis

Next, let's look at the status values that appear in the log. We want to know which status values appear in the data and how many times.

In [11]:
from pyspark.sql.functions import count

HTTP_status = weblog_parsed.groupBy('status').agg(count('status').alias('status_count')).sort(('status'))

display(HTTP_status)

status,status_count
200,1398988
302,26497
304,134146
400,10
403,171
404,10056
500,3
501,27


## 3. Frequent Hosts

Find the hosts that have accessed the server frequently. As with the response code analysis We then filter the result based on the count of accesses by each host. Then, we select the 'host' column and show few elements from the result.

In [13]:
from pyspark.sql.functions import desc

host_frequency = weblog_parsed.groupBy('host').agg(count('host').alias('host_frequency')).sort(desc('host_frequency'))

display(host_frequency)

host,host_frequency
edams.ksc.nasa.gov,6530
piweba4y.prodigy.com,4846
163.206.89.4,4791
piweba5y.prodigy.com,4607
piweba3y.prodigy.com,4416
www-d1.proxy.aol.com,3889
www-b2.proxy.aol.com,3534
www-b3.proxy.aol.com,3463
www-c5.proxy.aol.com,3423
www-b5.proxy.aol.com,3411


##4. Find the top paths/request-strings (URIs) in the log

In [15]:
request_stringCount = weblog_parsed.groupBy('request_string').agg(count('request_string').alias('request_stringCount')).sort(desc('request_stringCount'))

display(request_stringCount)

request_string,request_stringCount
/images/NASA-logosmall.gif,97275
/images/KSC-logosmall.gif,75283
/images/MOSAIC-logosmall.gif,67356
/images/USA-logosmall.gif,66975
/images/WORLD-logosmall.gif,66351
/images/ksclogo-medium.gif,62670
/ksc.html,43618
/history/apollo/images/apollo-logo1.gif,37806
/images/launch-logo.gif,35119
/,30105


## 5. Find top ten error paths which did not have return code 200?

In [17]:
Error_Paths = weblog_parsed.filter("status!=200").groupBy('request_string','status').agg(count('request_string').alias('request_string_error')).sort(desc('request_string_error')).limit(10)

display(Error_Paths)

request_string,status,request_string_error
/images/NASA-logosmall.gif,304,19072
/images/KSC-logosmall.gif,304,11328
/images/MOSAIC-logosmall.gif,304,8617
/images/USA-logosmall.gif,304,8565
/images/WORLD-logosmall.gif,304,8360
/images/ksclogo-medium.gif,304,7722
/history/apollo/images/apollo-logo1.gif,304,4355
/shuttle/countdown/images/countclock.gif,304,4227
/images/launch-logo.gif,304,4178
/,304,3605


##6. Number of unique hosts


How many unique hosts are there in the entire log?

In [19]:
unique_hosts = weblog_parsed.select('host').distinct().count()

print(unique_hosts)

75060

## 7. Number of Unique Daily Hosts

In [21]:
from pyspark.sql.functions import substring, countDistinct

unique_hosts_perday = weblog_parsed.groupBy(substring('dateTime',1,11).alias('date')).agg(countDistinct('host')).sort('date')

display(unique_hosts_perday)

date,count(DISTINCT host)
01/Aug/1995,2582
03/Aug/1995,3222
04/Aug/1995,4191
05/Aug/1995,2502
06/Aug/1995,2538
07/Aug/1995,4108
08/Aug/1995,4406
09/Aug/1995,4317
10/Aug/1995,4523
11/Aug/1995,4346


##8. Number of Daily Requests per Unique Host

In [23]:
eachHost_requests_perday = weblog_parsed.groupBy('host',substring('dateTime',1,11).alias('day')).agg(count('host').alias('NumberofRequest')).sort('day')

display(eachHost_requests_perday)

host,day,NumberofRequest
asd01-18.dial.xs4all.nl,01/Aug/1995,1
bb.iu.net,01/Aug/1995,1
l43jo.jsc.nasa.gov,01/Aug/1995,25
n1031659.ksc.nasa.gov,01/Aug/1995,6
131.125.1.205,01/Aug/1995,122
claude.millipore.com,01/Aug/1995,1
hvsun21.mdc.com,01/Aug/1995,19
centauri.tksc.nasda.go.jp,01/Aug/1995,13
ip133.san-francisco.ca.interramp.com,01/Aug/1995,6
163.205.3.51,01/Aug/1995,6


## 9. Counting 404 Response Codes

In [25]:
status404 = weblog_parsed.filter("status=404").groupBy('status').agg(count('status').alias('host_frequency')).sort(desc('host_frequency'))

display(status404)


status,host_frequency
404,10056


## 10. Find top twenty request_string/paths that generate the most 404 errors.

In [27]:
status404_path = weblog_parsed.filter("status=404").groupBy('request_string').agg(count('request_string').alias('request_stringCount')).sort(desc('request_stringCount')).limit(20)

display(status404_path)

request_string,request_stringCount
/pub/winvn/readme.txt,1337
/pub/winvn/release.txt,1185
/shuttle/missions/STS-69/mission-STS-69.html,682
/images/nasa-logo.gif,319
/shuttle/missions/sts-68/ksc-upclose.gif,251
/elv/DELTA/uncons.htm,209
/history/apollo/sa-1/sa-1-patch-small.gif,200
/://spacelink.msfc.nasa.gov,166
/images/crawlerway-logo.gif,160
/history/apollo/a-001/a-001-patch-small.gif,154


## 11. Visualizing the 404 Errors by Day

In [29]:
status404_day = weblog_parsed.filter("status=404").groupBy(substring('dateTime',1,11).alias('date')).agg(count('dateTime').alias('status404_day')).sort('date')

display(status404_day)

date,status404_day
01/Aug/1995,243
03/Aug/1995,304
04/Aug/1995,346
05/Aug/1995,236
06/Aug/1995,373
07/Aug/1995,537
08/Aug/1995,391
09/Aug/1995,279
10/Aug/1995,315
11/Aug/1995,263


## 12. Top Five Days for 404 Errors

In [31]:
status404_day_top5 = weblog_parsed.filter("status=404").groupBy(substring('dateTime',1,11).alias('date')).agg(count('dateTime').alias('status404_day')).sort(desc('status404_day')).limit(5)

display(status404_day_top5)

date,status404_day
30/Aug/1995,571
07/Aug/1995,537
31/Aug/1995,526
24/Aug/1995,420
29/Aug/1995,420
